## COCO2labelme

Converts COCO JSON file to labelme JSON so that annotations can be modified easily

@author ghowa

In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
import json
from shutil import copy2
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [3]:
# annFolder = "train-coco"
# annFile=os.path.join(annFolder, "annotations.json")
# annFile = ("../Fsz0hKLEPdc.mkv.json")
valFile = ("../annotations/val.json")
testFile = ("../annotations/test.json")
trainFile = ("../annotations/train.json")


In [4]:
# initialize COCO api for instance annotations
coco_val=COCO(valFile)
coco_test=COCO(testFile)
coco_train=COCO(trainFile)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [7]:
# display COCO categories and supercategories
cats = coco_val.loadCats(coco_val.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

COCO categories: 
kreuz adler flagge_ukr flagge_upa ukraine hakenkreuz ss_rune bandera georgsband hammer&sichel Wolfsangel oun swoboda falanga nato flagge_ru eu



In [16]:
# sort all images into category folders
folders = ["val_images", "test_images", "train_images"]
count = 0
for coco in [coco_val, coco_test, coco_train]:
    if not os.path.exists(folders[count]):
        os.mkdir(folders[count])
    for name in nms:
        catIds = coco.getCatIds(catNms=[name]) # [name] is used to prevent lookup bug
        imgIds = coco.getImgIds(catIds=catIds)
        if not os.path.exists(folders[count]+"/"+name):
            os.mkdir(folders[count]+"/"+name)
        for imId in imgIds:
            # copy images
            img = coco.loadImgs(imId)
            copy2("../annotations/"+folders[count]+"/"+img[0]["file_name"], folders[count]+"/"+name+"/"+img[0]["file_name"])

            # write labelme annotations
            output = {"version": "4.5.6", "flags": {}}
            annIds = coco.getAnnIds(imgIds=imId, catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
            shapes = []
            for ann in anns:
                points = []
                for index in range(len(ann["segmentation"][0])):
                    if index%2 == 0:
                        points.append([ann["segmentation"][0][index],ann["segmentation"][0][index+1]])
                shapes.append({"label": name, "points": points, "group_id": None, "shape_type": "polygon", "flags":{}})

            output["shapes"] = shapes
            output["imagePath"] = img[0]["file_name"]
            output["imageData"] = None
            output["imageHeight"] = img[0]["height"]
            output["imageWidth"] = img[0]["width"]

            with open(folders[count]+"/"+name+"/"+img[0]["file_name"].rsplit(".",1)[0]+".json", 'w') as outfile:
                json.dump(output, outfile)  
    count += 1